In [1]:
import pandas as pd
import numpy as np

In [1]:
#reading datafile
df = pd.read_csv('movies.csv',low_memory=False)

NameError: name 'pd' is not defined

In [50]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [52]:
df.shape

(45466, 24)

<h3>Taking overview column as a feature

In [53]:
df = df[['original_title','overview']].head(1000)
# df.head()

In [54]:
df.isnull().sum()

original_title     0
overview          12
dtype: int64

In [55]:
df.dropna(inplace=True)
df.isnull().sum()

original_title    0
overview          0
dtype: int64

In [8]:
df.isnull().sum()


original_title    0
overview          0
dtype: int64

In [56]:
df.shape

(988, 2)

<h2>importing 'TfidfVectorizer' to calculate TF-IDF score for overview

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

<h3>stop_words is use to neglect words like 'is','the','of' etc

In [11]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1,3),stop_words='english')

In [12]:
matrix = tf.fit_transform(df['overview'])


<h4>matrix consist of documents(overviews in this case) and word count in each document

In [32]:
df.shape,matrix.shape

((988, 2), (988, 63563))

<h2>Trying different methods for similarity

In [60]:
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel,sigmoid_kernel
from sklearn.metrics import pairwise_distances

In [61]:
cos_sim = cosine_similarity(matrix)

In [62]:
sig_sim = sigmoid_kernel(matrix)

In [63]:
lin_sim = linear_kernel(matrix)

In [64]:
sim_dist = pairwise_distances(matrix,metric='euclidean')
sim_dist2 = pairwise_distances(matrix,metric='manhattan')

In [40]:
pp=pd.DataFrame(cos_sim ,index=df['original_title'],columns=df['original_title'])

In [41]:
indices = pd.Series(df.index, index=df['original_title'])

In [42]:
def getMovie(name,sim_mat,k):
    idx = indices[name]
    score = list(enumerate(sim_mat[idx]))
    score = sorted(score , key=lambda x:x[1],reverse=True)
    movies_idx =  [i[0] for i in score[1:k]]
    return df.iloc[movies_idx]['original_title']


In [65]:
mat={'cosine':cos_sim,'sigmoid':sig_sim,'linear':lin_sim,'eculidean':sim_dist,'manhattan':sim_dist2}

In [80]:
for name,m in mat.items():
    print(f'{name} similarity \nResult :\n{"-"*50}\n{getMovie("Toy Story",m,3)}\n{"-"*50}\n\n')

cosine similarity 
Result :
--------------------------------------------------
485               Malice
448    For Love or Money
Name: original_title, dtype: object
--------------------------------------------------


sigmoid similarity 
Result :
--------------------------------------------------
485               Malice
448    For Love or Money
Name: original_title, dtype: object
--------------------------------------------------


linear similarity 
Result :
--------------------------------------------------
485               Malice
448    For Love or Money
Name: original_title, dtype: object
--------------------------------------------------


eculidean similarity 
Result :
--------------------------------------------------
912    The Reluctant Debutante
457       A Good Man in Africa
Name: original_title, dtype: object
--------------------------------------------------


manhattan similarity 
Result :
--------------------------------------------------
734    Vermont Is for Lovers
9